Large time window 25ns

small time window 3.125ns

In [1]:
import ROOT
import time

Welcome to JupyROOT 6.22/02


In [2]:
class noise_generator:
    def __init__(self, treshold, channel_count):
        noise = ROOT.TF1("noise_gauss", "TMath::Gaus(x, 0, 5)", 0, 1000)
        # Question: How sould the normalization be made, from -1000 to 1000 or from 0 to 1000?
        self._mean = channel_count * noise.Integral(treshold, 1000)/noise.Integral(0, 1000)
        self._prng = ROOT.TRandom3(time.time_ns())

    def generate(self):
        total = self._prng.Poisson(self._mean)
        narrow = 0
        for i in range(total):
            if self._prng.Uniform(25) < 3.125:
                narrow += 1
        estimate = 3.125 * (total - narrow)/(25 - 3.125)
        return narrow, estimate
    
    
class signal_generator:
    def __init__(self):
        self._prng = ROOT.TRandom3(time.time_ns())
    
    def generate(self, treshold, expected_voltage, expected_count):
        signal_count = self._prng.Poisson(expected_count)
        above_treshold_count = 0
        for i in range(signal_count):
            if self._prng.Landau(expected_voltage, expected_voltage/4) > treshold:
                above_treshold_count += 1
        return signal_count, above_treshold_count

### Example

In [3]:
# Create a noise generator by passing treshold and channel count as paramaters
my_noise_generator = noise_generator(20, 3.6 * 1e6)

# The first return value is the exact number of noise activated channels in the small window and
# the second return value is the estimated number of noise activated channels in the small window.
exact, estimate = my_noise_generator.generate()

print(exact, estimate)

# Create a signal generator
my_signal_generator = signal_generator()

# the first paramater is the treshold, the second paramater is the expected peak voltage and the third
# paramater is the expected number of particles to reach the detector.
# The first return value is the number of particles that reached the detector and the second return
# value is the number of activated channels.
all_count, above_treshold = my_signal_generator.generate(20, 100, 30)

print(all_count, above_treshold)

28 32.57142857142857
23 23
